# 定性評価

# -----------初期設定-----------

In [1]:
# -*- coding: utf-8 -*-
# python3
#
import argparse
import gzip
import math
import numpy
import re
import sys
import numpy as np
from copy import deepcopy
import codecs
from statistics import mean, median,variance,stdev
import pandas as pd
import pandas_profiling as pdp

In [2]:
from sklearn.externals import joblib

newvec = joblib.load("./word2vec.pkl")

# Qualitative_Evaluation（各次元の外れ値（ =分散が大きい）の単語Top5）

In [3]:
newvec_word = list(newvec.keys())

In [4]:
f_word = pd.DataFrame(newvec_word, columns=['word'])

In [5]:
f_word

,word
0,</s>
1,の
2,、
3,に
4,は
5,する
6,を
7,た
8,が
9,て


In [6]:
newvec_dimention = list(newvec.values())

In [7]:
# limitまでの次元まで (全単語の100次元までを集計)
var_per_dimention = {}
limit_num = 100
for v in range(len(newvec_dimention)):
    for j in range(len(newvec_dimention[v][0])):
        if j < limit_num:
            try:
                var_per_dimention[j].append(newvec_dimention[v][0][j])
            except:
                var_per_dimention[j] = [newvec_dimention[v][0][j]]
        else:
            break

## 単語操作

In [29]:
# 特定の次元の平均を求める
x = list(var_per_dimention.values())[99]
mean_x = np.average(x)

In [30]:
# 求めた平均から各単語の分散を求める
var_x = [math.sqrt((mean_x - x[i])**2) for i in range(len(x))]

# 分散の大きさ順にソートし，indexを返す
index_x_sorted = sorted(range(len(var_x)), key=lambda k: var_x[k], reverse=True)

In [31]:
# indexの上位5個
target = index_x_sorted[:5]
# indexの上位5個の単語を返す
f_word.iloc[target]

,word
85842,ユナイテッド・シネマ
49439,LORD
67488,水泳場
116118,ウォーターパーク
55247,総合運動公園


## d = 1

word  
41493	美しい村  
75994	日米和親条約  
75479	ゴルディアヌス  
77947	Cd  
137385	冬の夜  

## d = 2

word  
50780	青函  
98774	舞浜  
70466	1301  
85760	臨海副都心  
184110	1515  

## d = 3
	word
77460	前田山  
6086	"|  
78581	クマネズミ  
63078	葬儀委員長  
33510	旧自治体  

## d = 4

	word  
68675	爻  
131393	姿川  
46437	劇場盤  
82163	頗  
108423	仙塩  

## d = 30

	word  
85479	トップイーストリーグ  
74327	後日発表  
90075	重連運転  
82178	オーストラリアン・ベースボールリーグ  
74099	千葉県流山市  

## d = 50
	word  
34418	秋田県秋田市  
17940	梁山泊  
90759	耆  
49051	湘軍  
98838	喜平  

## d = 99
  
word  
85842	ユナイテッド・シネマ  
49439	LORD  
67488	水泳場  
116118	ウォーターパーク  
55247	総合運動公園  

In [27]:
# f_dimention = pd.DataFrame(list(var_per_dimention.values()))

In [ ]:
# pdp.ProfileReport(f_dimention.T)

# -----------ある単語の類似する単語を挙げる-----------

In [ ]:
def checkSim_by_word(vecs, word):
    # 閾値の設定
    negative = False # Falseなら似た単語を候補で上げる
    threshold = 0.5 # -1なら閾値固定
    border_positive = threshold if threshold > 0 else 0.8
    border_negative = threshold if threshold > 0 else 0.3

    # 候補数の設定
    max_candidates = 20
    candidates = {}
    
    # wordの設定確認
    if not word:
        raise Exception("word is missing")

    # wordがモデルにない場合，
    if word not in vecs:
        raise Exception("Sorry, this word is not registered in model.")

    # ベクトルの設定
    w_vec = vecs[word]
    for w in vecs:
        try:
            if w_vec.shape != vecs[w].shape:
                raise Exception("size not match")
            s = similarity(w_vec, vecs[w])
        except Exception as ex:
            print(w + " is not valid word.")
            continue

        if negative and s <= border_negative:
            candidates[w] = s
            if len(candidates) % 5 == 0:
                border_negative -= 0.05
        elif not negative and s >= border_positive:
            candidates[w] = s
            if len(candidates) % 5 == 0:
                border_positive += 0.05

        if len(candidates) > max_candidates:
            break
    
    # 類義語算出
    sorted_candidates = sorted(candidates, key=candidates.get, reverse=not negative)
    for c in sorted_candidates:
        print("{0}, {1}".format(c, candidates[c]))

In [ ]:
word = 'on'

In [ ]:
# 初期vecの場合，
checkSim_by_word(vecs_wordVecs, word)
print(' ')
# newvecの場合，
checkSim_by_word(vecs_new_vec, word)

# -----------WordSim評価-----------

In [ ]:
# word2vec
vecs_wordVecs = wordVecs

In [ ]:
# newvec
vecs_new_vec = new_vec

In [ ]:
def checkSim(v1, v2):
    if v1 not in lexicon: # 注目単語がwordnetに含まれない場合，
        print("v1(={})はwordnetのkeyに存在しません".format(v1))
        wordNeighbours_1 = set()
    else:
        # 注目単語の同義語リストとword2vecのkeyリストと重複する単語リスト（更新対象か？）
        wordNeighbours_1 = set(lexicon[v1]).intersection(set(wordVecs.keys()))
        print('v1(={})における更新対象のneighbour数 : {}'.format(v1, len(wordNeighbours_1)))
        
    # neighboursの中で注目単語と類似度が低いもの
    ave_1_word2vec = np.zeros_like(wordVecs[v1])  #neighboursの平均ベクトル
    ave_1_retrofit = np.zeros_like(wordVecs[v1])
    for neighbour in wordNeighbours_1:
        print('v1とneighbour(={}) : {} -> {}'.format(neighbour, 
                                                                   similarity(wordVecs[neighbour], wordVecs[v1]), 
                                                                   similarity(vecs_new_vec[neighbour], vecs_new_vec[v1])))
        ave_1_word2vec += wordVecs[neighbour]
        ave_1_retrofit += vecs_new_vec[neighbour]
    print('v1とneighboursの平均ベクトル : {} -> {}'.format(similarity(ave_1_word2vec/len(wordNeighbours_1), wordVecs[v1]), 
                                                              similarity(ave_1_retrofit/len(wordNeighbours_1), vecs_new_vec[v1])))
    print(" ")
    
    if v2 not in lexicon:
        print("v2(={})はwordnetのkeyに存在しません".format(v2))
        wordNeighbours_2 = set()
    else:
        wordNeighbours_2 = set(lexicon[v2]).intersection(set(wordVecs.keys()))
        print('v2(={})における更新対象のneighbour数 : {}'.format(v2, len(wordNeighbours_2))) # 更新対象数
    
    # neighboursの中で注目単語と類似度が低いもの
    ave_2_word2vec = np.zeros_like(wordVecs[v2])  #neighboursの平均ベクトル
    ave_2_retrofit = np.zeros_like(wordVecs[v2])
    for neighbour in wordNeighbours_2:
        print('v2とneighbour(={}) : {} -> {}'.format(neighbour, 
                                                                   similarity(wordVecs[neighbour], wordVecs[v2]), 
                                                                   similarity(vecs_new_vec[neighbour], vecs_new_vec[v2])))
        ave_2_word2vec += wordVecs[neighbour]
        ave_2_retrofit += vecs_new_vec[neighbour]
    print('v2とneighboursの平均ベクトル : {} -> {}'.format(similarity(ave_2_word2vec/len(wordNeighbours_2), wordVecs[v2]), 
                                                              similarity(ave_2_retrofit/len(wordNeighbours_2), vecs_new_vec[v2])))
    print(" ")
    
    # v1とv2における更新対象のneighbourに重複があるか
    print('更新対象における重複 : {}'.format(set(wordNeighbours_1).intersection(set(wordNeighbours_2))))
    print('更新対象における重複数 : {}'.format(len(set(wordNeighbours_1).intersection(set(wordNeighbours_2)))))
    print(" ")

    """v1とv2のword2vecとretrofittingにおける類似度"""
    try:
        print("word2vec : {}".format(similarity(vecs_wordVecs[v1], vecs_wordVecs[v2])))
    except:
        print('error')

    try:
        print("retrofitting : {}".format(similarity(vecs_new_vec[v1], vecs_new_vec[v2])))
    except:
        print('error')

    print(" ")
    """同一単語におけるword2vecとretrofittingの類似度"""
    try:
        print("v1 : {}".format(similarity(vecs_wordVecs[v1], vecs_new_vec[v1])))
    except:
        print('error')

    try:
        print("v2 : {}".format(similarity(vecs_wordVecs[v2], vecs_new_vec[v2])))
    except:
        print('error')

## ・WordSimのテスト

In [ ]:
v1 = 'フットボール'
v2 = 'サッカー'
# checkSim(v1, v2)

# -----------wordAnalogy評価-----------

In [ ]:
def checkAnalogy(vecs, w_vec):  
    negative = False # Falseなら似た単語を候補で上げる
    threshold = 0.3 # -1なら閾値固定

    # 閾値の設定
    border_positive = threshold if threshold > 0 else 0.9
    border_negative = threshold if threshold > 0 else 0.2
    print('{} < thd < {}'.format(border_negative, border_positive))

    # 候補数の設定
    max_candidates = 20
    candidates = {}

    for w in vecs:
        try:
            if w_vec.shape != vecs[w].shape:
                raise Exception("size not match")
            s = similarity(w_vec, vecs[w])
        except Exception as ex:
            print(w + " is not valid word.")
            continue

        if negative and s <= border_negative:
            candidates[w] = s
            if len(candidates) % 5 == 0:
                border_negative -= 0.05
        elif not negative and s >= border_positive:
            candidates[w] = s
            if len(candidates) % 5 == 0:
                border_positive += 0.05

        if len(candidates) > max_candidates:
            break

    # 類義語算出
    sorted_candidates = sorted(candidates, key=candidates.get, reverse=not negative)
    for c in sorted_candidates:
        print("{0}, {1}".format(c, candidates[c]))

## ・「v1 + v2 - v3」と「v4」の類似度算出

In [ ]:
v1 = '兄'
v2 = '姉'
v3 = '祖父'
v4 = '祖母'

In [ ]:
if v1 not in lexicon:
    print("v1 not found error in dict")
if v2 not in lexicon:
    print("v2 not found error in dict")
if v3 not in lexicon:
    print("v3 not found error in dict")
if v4 not in lexicon:
    print("v4 not found error in dict")

try:
    print('word2vec : {}'.format(similarity(vecs_wordVecs[v1] + vecs_wordVecs[v2] - vecs_wordVecs[v3], vecs_wordVecs[v4])))
except:
    print('error')

try:
    print('retrofitting : {}'.format(similarity(vecs_new_vec[v1] + vecs_new_vec[v2] - vecs_new_vec[v3], vecs_new_vec[v4])))
except:
    print('error')

## ・「v1 + v2 - v3」と近い単語を挙げる→「v4」が結果に出るか

In [ ]:
# 初期vecの場合，
checkAnalogy(vecs_wordVecs, vecs_wordVecs[v1] + vecs_wordVecs[v2] - vecs_wordVecs[v3])
print(' ')
# newvecの場合，
checkAnalogy(vecs_new_vec, vecs_new_vec[v1] + vecs_new_vec[v2] - vecs_new_vec[v3])